# Dynamic Programming
## Matrix Chain Multiplacation

Given a list of dimensions of compatible matrices calculate the most effienct ordering of multiplications of the given matrices.

To do this we will create a n-1 * n-1 matrix where n is the number of dimensions given to us. The goal is to calculate the cell [1 , n-1] as this will hold the smallest number of multiplications needed to multiply all the compatible matrices. We will also store the matrices that were multiplied behind the scenes so we can retrace the optimal multiplication of matrices. 

Here is the method to do so.

1. Create a n-1 * n-1 matrix where n in the number of dimensions given to you.
- We only need half of the matrix that we are creating to calculate the solution.  
2. Initialize the diagonal values to all 0's
3. Then we will calculate all 2 adjacent matrices being multiplied together starting with 1,2 then 2,3 
4. We will then procede with 3 adjacent matrices working our way up to all matrices being multipled together
5. To calculate the cell C[i,j] when 3 matrices or more are being multiplied together. We must consider the min value of these choices from k = i to j -1 we elvulate C[i,k] + C[k+1,j] + p(i-1) * p(k) * p(j) where p(x) is the xth element in the list of dimensions.
6. We will also store the choosen min ordering for the cell C[i,j] so we can retrace the solution later.
7. The final solution will be at [1,n-1]

The tables below demonstrate the progress of the algorithm.

In [59]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

def style_specific_cell(df, row, col, c):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    df_styler.iloc[row, col] = 'color: black;background-color: ' + c
    return df_styler

def style_case1_fails(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    #one above
    df_styler.iloc[row-1, col] = 'color: black; background-color: lightgreen'
    #weight
    df_styler.iloc[row, 0] = 'color: black; background-color: firebrick; font-weight:bold'
    #value
    df_styler.iloc[row, 1] = 'color: black; background-color: firebrick; font-weight:bold'
    #column
    df_styler.iloc[0, col] = 'color: black; background-color: firebrick; font-weight:bold'
    return df_styler

def style_case1(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    # one above
    df_styler.iloc[row-1, col] = 'color: black; background-color: firebrick'
    #weight 
    df_styler.iloc[row, 0] = 'color: black; background-color: lightgreen; font-weight:bold'
    #value 
    df_styler.iloc[row, 1] = 'color: black; background-color: lightgreen; font-weight:bold'
    #column value
    df_styler.iloc[0, col] = 'color: black; background-color: lightgreen; font-weight:bold'
    #case 1 added value
    df_styler.iloc[row-1,col-df.iloc[row, 0]] = 'color: black; background-color: lightgreen; font-weight:bold' 
    #column
    df_styler.iloc[0, col] = 'color: black; background-color: lightgreen; font-weight:bold'

    return df_styler

def style_case2(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    # one above
    df_styler.iloc[row-1, col] = 'color: black; background-color: lightgreen'
    #weight 
    df_styler.iloc[row, 0] = 'color: black; background-color: darkorange; font-weight:bold'
    #value 
    df_styler.iloc[row, 1] = 'color: black; background-color: darkorange; font-weight:bold'
    #column val
    df_styler.iloc[0, col] = 'color: black; background-color: darkorange; font-weight:bold'
    #case 1 added value
    df_styler.iloc[row-1,col-df.iloc[row, 0]] = 'color: black; background-color: darkorange; font-weight:bold'
    #column
    df_styler.iloc[0, col] = 'color: black; background-color: darkorange; font-weight:bold'

    return df_styler

def style_traceback(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    x = row
    y = col
    while df.iloc[x, y] != 0:
        if df.iloc[x-1, y] == df.iloc[x,y]:
            df_styler.iloc[x-1, y] = 'color: black; background-color: gold'
            x -= 1
        else:
            df_styler.iloc[x-1, y-df.iloc[x,0]] = 'color: black; background-color: gold'
            df_styler.iloc[x,3] = 'color: black; background-color: yellowgreen'
            y -= df.iloc[x,0]
            x -= 1
    return df_styler

In [60]:
################################# DEFINE [WEIGHT, VALUE] FOR EACH OBJECT HERE #################################
dimensions = [30,60,170,45,160,30] #leave first 2 items
#############################################################################################################
num_matrices = len(dimensions) -1
df = pd.DataFrame(np.zeros((num_matrices,num_matrices)) , columns=[x for x in range(1,num_matrices+1)]).astype(int)
df.index = df.index + 1
for i in range(1, num_matrices):
    df.iloc[i:, i-1] = ['-' for x in range(i, num_matrices)]

df.style




,1,2,3,4,5
1,0,0,0,0,0
2,-,0,0,0,0
3,-,-,0,0,0
4,-,-,-,0,0
5,-,-,-,-,0


In [63]:
#calculating 2 matrices multipled together
d = {}
for i in range(1,num_matrices):
    v = df.loc[i,i+1] = dimensions[i-1] * dimensions[i] * dimensions[i+1]
    print(f'Value for cell {i},{i+1}')
    print(f'c[{i},{i+1}] = p{i-1}*p{i}*p{i+1} = {dimensions[i-1]}*{dimensions[i]}*{dimensions[i+1]} = {v}')
# calculating all value for all 3 or more multiplied matrices
for k in range(2, num_matrices):
    for i in range(1, num_matrices - k + 1):
        j = i + k
        value = []
        print(f'Values considered for cell {i},{j}')
        for x in range(i,j):
            v = df.loc[i,x] + df.loc[x+1,j] + dimensions[i-1] * dimensions[x] * dimensions[j]
            print(f'c[{i},{x}] + c[{x+1},{j}] +  p{i-1}*p{x}*p{j} = {df.loc[i,x]} + {df.loc[x+1,j]} + {dimensions[i-1]}*{dimensions[x]}*{dimensions[j]} = {v}')
            value.append([v,f'{i}{x}{x+1}{j}'])
        df.loc[i,j] = min(value)[0]
        d[f'{i}{j}'] = min(value)[1]
display(df.style)


Value for cell 1,2
c[1,2] = p0*p1*p2 = 30*60*170 = 306000
Value for cell 2,3
c[2,3] = p1*p2*p3 = 60*170*45 = 459000
Value for cell 3,4
c[3,4] = p2*p3*p4 = 170*45*160 = 1224000
Value for cell 4,5
c[4,5] = p3*p4*p5 = 45*160*30 = 216000
Values considered for cell 1,3
c[1,1] + c[2,3] +  p0*p1*p3 = 0 + 459000 + 30*60*45 = 540000
c[1,2] + c[3,3] +  p0*p2*p3 = 306000 + 0 + 30*170*45 = 535500
Values considered for cell 2,4
c[2,2] + c[3,4] +  p1*p2*p4 = 0 + 1224000 + 60*170*160 = 2856000
c[2,3] + c[4,4] +  p1*p3*p4 = 459000 + 0 + 60*45*160 = 891000
Values considered for cell 3,5
c[3,3] + c[4,5] +  p2*p3*p5 = 0 + 216000 + 170*45*30 = 445500
c[3,4] + c[5,5] +  p2*p4*p5 = 1224000 + 0 + 170*160*30 = 2040000
Values considered for cell 1,4
c[1,1] + c[2,4] +  p0*p1*p4 = 0 + 891000 + 30*60*160 = 1179000
c[1,2] + c[3,4] +  p0*p2*p4 = 306000 + 1224000 + 30*170*160 = 2346000
c[1,3] + c[4,4] +  p0*p3*p4 = 535500 + 0 + 30*45*160 = 751500
Values considered for cell 2,5
c[2,2] + c[3,5] +  p1*p2*p5 = 0 + 44550

,1,2,3,4,5
1,0,306000,535500,751500,792000
2,-,0,459000,891000,751500
3,-,-,0,1224000,445500
4,-,-,-,0,216000
5,-,-,-,-,0


The final table above shows the least number of multiplications needed in the top right cell:

The solution is 792,000 multiplications which corresponds to matrix order ((A1* A2) (A3))(A4 * A5)
